# Deep Learning-Based Document Modeling for Persoonality Detection from Text
## テキストからの性格推定のためのディープラーニングベースド文書モデル
Navonil Majumder - Instituto Politecnico Nacional


## 0 - abstract
性格は個人の振る舞いや感情、興味と特徴の組み合わせである  
この性格というものは我々の人生に大きな影響を与える  
本論では、CNNを用いてstream-of-consciousness essayから性格特性を抽出する手法を提案する  
同じ構造を持った異なる5つのネットワークを学習することで、先行研究にある5つの性格特性を求める(Big Five Model)  
すべてのネットワークは二値分類器であり、正か負かの相関特性を予測する  
この目的を達成するため、CNNによる特徴抽出器を基にした新しい文書モデリング手法を開発した  
すなわち、エッセイから取ってきた文章を畳み込みフィルタに食わせてn-gram特徴ベクトルの形で文のモデルを得、それぞれのエッセイを文ベクトルの集合として表現した  
こうして得られたベクトルを、前処理であらかじめテキストから直接抽出したMairesse特徴と結びつけた。こうすることで手法が改善された  
エッセイから中立感情の文を破棄したらもっと改善される  
最終的な分類のため、この文書ベクトルをニューラルネットワークの隠れ層の一つに連結した  
その結果、5つすべての特性について既存研究よりもよい数値を示した  


## 1 - 先行研究
Five Factor Modelとしても知られるBig Fiveモデルはもっとも広く持ちいられている性格モデル  
プレーンテキストからの自動性格特定の先行研究として、エッセイのデータセットを収集したJames PennebakerとLaura King  
彼らはボランティアによって書かれたstream-of-consciousness essayを収集した

## 2 - 手法の概要
手法は、入力データの前処理とフィルタリング、特徴抽出、分類が含まれる  
特徴には以下の2つのタイプを用いる  
- 固定長文書レベルの文体的特徴
- 入力テキストの可変長な表現を組み合わせた単語レベルの意味的特徴  

この可変長の表現がCNNにブチ込まれる  
CNNでは、単語をn-gramに、n-gramを文に、文をドキュメント全体にというような手法を組み合わせ階層的に処理される  
得られた値はその後、最終的な分類に用いるための文書表現を形成するために、文書レベルの文体的特徴と組み合わされる  

手法は次のようなステップからなる:
1. 前処理  
文の分割, 小文字への変換といったデータの消去と統合を行う  
1. 文書単位での特徴抽出  
Mairesseベースラインの特徴セットを用いる
1. フィルタリング  
エッセイに含まれる文には性格に寄与しない文もある  
そのような文を意味的特徴の抽出において無視する  
理由は2つ
    - そのようなノイス表現は分類の性能を落とすため  
    - これらの文を削除することで入力のサイズを落とすことができ、結果に悪影響を及ぼさずに訓練時間を落とせるため  

よって、次のステップの前に寄与しない文を削除する  

1. 単語単位の特徴抽出  
個々の単語をワードエンベッティングにより連続ベクトル空間に表現する  
具体的にはword2vecを用いた  
これにより文書の可変特徴セットが得られ、文書はいくつかの固定長単語特徴ベクトルで表されるいくつかの文章集合として表現される    

1. 分類  
分類はCNNを用いて行われる  
初期レイヤはテキストを階層的に処理する  
それぞれの入力単語はword2vecを用いた固定長の特徴ベクトルとして表現され、文章は可変数の単語ベクトルとして表される  
いくつかのレイヤでは、この可変ベクトルが各文の固定長ベクトルに圧縮され、これは連続ベクトル空間に埋め込まれる文の一種となる  
このレベルになると、文書はいくつかの固定長の文埋め込みの集合として表現される  
そして、より深いレイヤでは可変長の文書ベクトルが固定長文書ベクトルに圧縮される  
この固定長特徴ベクトルはその後文書レベルの特徴と結合し、固定長の文書ベクトルを与え、最終的な分類に使用される  
  
  
単語ベクトルを文ベクトルに集約する際には、単語のn-gram特徴を用いる  
しかしながら、文ベクトルを文書ベクトルに集約する際には、文のn-gram特徴を用いることはしない  
試してみたけど75エポックでも収束しなかったからね  


## 3 - ネットワークの構造
BigFiveの性格特性のため、同じ構造を持つ5つの別個なニューラル分類器を訓練した  
ネットワークの処理フローは主に4つのステップからなる:

1. word2vecによる固定長単語埋め込みを入力データとして用いる、単語のベクトル化  
1. それぞれの文に含まれる単語の順序を固定長文ベクトルとする、文のベクトル化
1. 文の順序を文書ベクトルとする、文書のベクトル化
1. 文書ベクトルから分類結果(yes/no)を行う、分類  

したがって、ネットワークは次の7つのレイヤを含む  
入力(単語ベクトル化)  
畳み込み(文ベクトル化)  
maxプーリング(文ベクトル化)  
1-maxプーリング(文書ベクトル化)  
連結(文書ベクトル化)  
シグモイド活性化(分類)  
2つのニューロンソフトマックス出力(分類)  

### 3-1 入力 
データセットは文書の集合として表す  
文書$d$は文集合の順序であり、各文$s_i$は単語集合の順序であり、各単語$w_i$は単語埋め込みのような固定長の実数値ベクトルである  
入力層は$\mathbb{R}^{D\times S\times W\times E}$の四次元の実数行列であり、Dはデータセットである文書の数、Sはすべての文書での文の数の最大値、Wはすべての文中で最も長い単語の文字数、Eはワードエンベッティング後の長さ  
実装ではすべての文書が同じ数の文となるように、文が少ない文書にダミー文を埋め込んだ  

### 3-2 単語ベクトルの集合を文ベクトルに
各文からユニグラム、バイグラム、トリグラム特徴を抽出するため、3つの畳込みフィルタを用いる  
maxプーリング後、文ベクトルをこれらの畳込みふぃるたから得られた特徴ベクトルと結びつける  

#### 3-2-1 畳み込み
n-gram特徴を抽出するため、各文sについて$n x E$の大きさの畳み込みフィルタを適用した  
n=1,2,3に対し、200個のn-gram特徴量を用いた  
各nについて、行列sに適用される畳み込みフィルタは$F^{conv}_n\in\mathbb{R}^{200\times n\times E}$となる  
フィルタの出力には$B^{conv}_n\in\mathbb{R}^{200}$のバイアスを加えられ、与えられた文に対してn=1,2,3それぞれについて3つの特徴マップ$FM_n$が得られる  
また、非線形性を持たせるために正規化線形関数を特徴マップに適用する  

#### 3-2-2 Maxプーリング
次に、各特徴マップ$FM_n$についてMaxプーリングを適用し、さらにダウンサンプリングを用いることで特徴マップ$DFM_n$を得る  
さらにこれらを平滑化し、200次元の特徴ベクトルを得る  

#### 3-2-3 畳み込み
得られた3種のn-gramのベクトルを結合し、文$を表すベクトルs\in \mathbb{R}^{600}$を得る  
この特徴ベクトルを用い、文書内の文について畳み込みとMaxプーリングを適用する  
ネットワークのパラメータは文書内のすべての文間で共有される  
ただ、すべての文はダミーの単語を埋め込むなどして共通の長さに揃えるが、すべての文書の長さを揃えるためにダミー文を追加するということはしない  

### 3-3 文ベクトルの集合を文書ベクトルに
それぞれの文に処理を施した後、文書ベクトルはすべての文ベクトルを結合した可変長のベクトルとなる  
文書内における文のうち少なくとも一つにこのような特徴が含まれている場合、文書にはいくつかの特徴があると仮定する  
それぞれの文は600次元のベクトルとして表現される  
文書ベクトルを得るため、これら600の特徴量それぞれについて、文書内のすべての文における最大値を取る  
これにより、文書全体についての600次元実数ベクトル$d^{network}\in \mathbb{R}^{600}$が得られる  

### 3-4 文書ベクトルに文書レベルの特徴を加える
Francois Mairesseらは性格推定のための84の特徴からなる文書レベルでの特徴セットを開発した  
これはLIWC(Linguistic Inquiry and Word Count)特徴、MRC特徴、発話型特徴、韻律特徴から成り立つ  
このセットには、文ごとの単語数・平均単語数、また代名詞の総数、過去形、現在形、未来形、文字、音素、音節、疑問形、文書の主張といった特徴の例が含まれる  
これら84の特徴を$d^{Mairesse}$として文書ベクトル$d^{network}$に連結し、最終的に684次元の文書ベクトル$d=(d^{network},d^{Mairesse})\in\mathbb{R}^{684}$を得る  
また、このMairesseの特徴ベクトル$d^{Mairesse}$を評価の際のベースラインとして用いる  

### 3-5 分類
最終的な分類のため、サイズが200のFully Connected layerと、yes/noと対応するサイズが2のソフトマックス層からなる2層パーセプトロンを用いる  

#### 3-5-1 Fully connected layer
文書ベクトル$d\in\mathbb{R}^{684}$に行列$W^{fc}\in\mathbb{R}^{684\times200}$をかけ、さらにバイアスベクトル$B^{fc}\in\mathbb{R}^{200}$を加えることでベクトル$d^{fc}\in\mathbb{R}^{200}$を得る  
このベクトルにシグモイド活性化による非線形性を導入することで、結果を改善した  
活性化関数にReLUやらtanbを使ってみた実験もしたけど結果は低かったよ  
#### 3-5-2 ソフトマックス出力
文書がyes/noのクラスに属する確率を決定するため、ソフトマックス関数を用いる  
そのため、次のようなベクトルを構築する
$$(x_{yes},x_{no})=d^{fc}W^{sm}+ B^{sm}$$
ここで、$W^{sm}\in\mathbb{R}^{200}$であり、バイアス$B^{sm}\in\mathbb{R}^2$であり、クラス確率は以下のように計算する  
$$P(i|network\ parameters) = \frac{exp(x_i)}{exp(x_{yes})+exp(x_{no})}$$
$$for\in\{yes,no\}$$

### 3-4 学習
学習の目的関数には逆対数尤度を用いた  
はじめに、ネットワークのパラメータをランダムに初期化  
誤りの尺度としての逆対数尤度を最小化することによるパラメータの調整を行うため、Adadeltaアルゴリズムによる確率勾配降下法を適用した  
実験では、50エポックを超えたあたりで学習精度は98%を上回った  



## 4 実験結果
手法の評価を行うため、一般的によく使用されるデータセットを用いて性格推定手法の比較を行った  

### 4-1 データセット
データセットにはJames PennebakerとLaura Kingによるstream-of-consciousness essayデータセットを用いた  
これには2468件の匿名エッセイが含まれており、それぞれに筆者の性格特性(EXT, NEU, AGR, CON, OPN)がタグ付けされている  
このデータセットの中から"Err:508"とだけ記述された1件を除いた、2467件を実験に用いた  

### 4-2 実験設定
すべての実験で、訓練済みネットワークの評価は10分割交差検定を用いて行われた  

#### 4-2-1 前処理
ピリオドまたはクエスチョンマークを基に、テキストを文単位に区切る  
空白を区切りに文を単語に分割  
すべての文字を小文字に置き換え、ASCII以外の文字・数字・エクスクラメーションマーク・シングル/ダブルクオーテーションを文から除去  
データセットにはピリオドがないエッセイも含まれ、結果として長い文が残っている  
このような場合には、150単語を超える長さの"文"は20単語ごとの"文集合"として分割した(最後の部分は短くなる可能性がある)  

#### 4-2-2 文書レベル特徴の抽出
Mairesseらのライブラリを用いそれぞれの文書から84のMairesse特徴を抽出する  

#### 4-2-3 文のフィルタリング
関連する文には感情語があると仮定した  
文書レベル特徴の抽出後、word2vec特徴を抽出する前に、感情語を持たない文章(ニュートラルな文章？)をすべて除外した  
このような感情語を得るため、NRC感情レキシコンを用いた  
NRCには10種の属性(怒り・予感・嫌悪・恐怖・喜び・嫌い・好き・悲しみ・驚き・信頼)がタグ付けされた14182語の感情語が含まれている    
この10種の属性のうち、少なくともひとつを持っている単語は感情語であるとして考え、そのような単語は6468単語程度であった  
よって、文がこれら6468単語のいずれも含んでいない場合、word2vec特徴を抽出する前に除外した  
今回のデータセットでは、すべてのエッセイが少なくともひとつの感情語を含んでいた  
また、恣意的な除外でなくランダムにエッセイから半分の単語を除外した場合についても実験してみた  
すると、フィルタリングを全く行わなかった場合と比較して結果が改善されたが、この事実については説明できんしよくわからん  
前述したよう感情的に中立な文を取り除くと、結果はさらに改善され、5つすべての特性において最良の結果を残した  
学習時間についても、33.3%の改善がみられた  

#### 4-2-4 単語レベル特徴の抽出
word2vecエンベッティングを用いて単語を300次元のベクトルに変換した  
もし単語がリスト(？)に見つからない場合、300すべての座標を[-0.25,0.25]の連続一様分布からランダムに割り当てた

#### 4-2-5 単語nグラムベースライン
ベースラインの特徴セットとして、データセットにおける頻出単語のユニグラム・バイグラム・トリグラムそれぞれ10000を組み合わせた30000の特徴を用いた  
Scikit-learnライブラリを用いてこれらの特徴を全文書から抽出した  

#### 4-2-6 分類
3つの分類器を用いて実験を行った  
表1のMLPと表記された箇所では、図１に示すネットワークを用いた  
このネットワークは、CNNと共に訓練された1層の隠れ層を持つMLPである  
また、SVMと表記された箇所では、まず図1に示すようなネットワークの学習を行い、データセット内のそれぞれの文書に対応する文書ベクトルを得た  
そして、これらのベクトルを次数3の多項式カーネルを用いたSVMにぶちこんで学習した  
sMLP/MPと表記された箇所では、同様の方法でベクトルd(maxプーリングレイヤ)を用い、図1の最後の２つのレイヤと同じ構成のスタンドアロンMLPを学習した(50エポック)(図1の1-maxプールレイヤの出力を入力として用いた)  
別の実験では、ベクトルdの代わりにfully connectedレイヤをスタンドアロンMLPに食わせたものをsMLP/FCとして表1に表記した  
CNNを用いないベースライン実験では、線形SVMのみを用いた  

